In [1]:
from pymongo import MongoClient
import pandas as pd
from bson import ObjectId
from database import officesDataFrame

client = MongoClient()
db = client['companies']

In [10]:
#Create dataframe containing all the offices of the startup_tech companies
#Create dataframe containing all the offices available for renting

df_startups = officesDataFrame(db,{'isTech':True})
#df_available_offices = officesDataFrame(db,{'isAvailable':True})

#Take the 3 cities with highest number of tech startups
print('City to create my company: ')
selected_city = df_startups.city
df_startups.city.value_counts()

City to create my company: 


Cambridge              9
Redwood City           5
San Francisco          5
South San Francisco    4
Seattle                4
                      ..
Redwood Shores         1
Paris                  1
New Delhi              1
Arvada                 1
King of Prussia        1
Name: city, Length: 84, dtype: int64

In [16]:
#Keep only those offices in the desired cities
df_available_offices = df_available_offices[df_available_offices.
                                            apply(lambda s:s['city'] in selected_cities, axis = 1)]
df_startups = df_startups[df_startups.
                          apply(lambda s:s['city'] in selected_cities, axis = 1)]

db['startups'].insert_many(df_startups.T.to_dict().values())
db['rentings'].insert_many(df_available_offices.T.to_dict().values())

In [18]:
#Geolocalization
from location import invertCoordinates

#First invert coordinates [long,lat]
invertCoordinates(db['startups'])
invertCoordinates(db['rentings'])

In [24]:
#Map with tech startups and available offices in San Francisco

import folium
#latitude, longitude
m =folium.Map(location=[37.773972, -122.431297],zoom_start=13)

for office in db['city_study'].find():
    if office['classification'] == 'Tech Startup':
        folium.Marker(office['coordinates'],radius=2,
                        icon=folium.Icon(icon='android',prefix='fa',color='darkblue')).add_to(m)
    elif office['classification'] == 'Renting':
        folium.Marker(office['coordinates'],radius=2,
                        icon=folium.Icon(icon='home',prefix='fa',color='red')).add_to(m)
    elif office['classification'] == 'Starbucks':
        folium.Marker(office['coordinates'],radius=2,
                        icon=folium.Icon(icon='coffee',prefix='fa',color='darkgreen')).add_to(m)


m.save('startups_map')
m

In [82]:
#Time to find the starbucks
from dotenv import load_dotenv
load_dotenv()

#latitude longitude
def starbucksToMongo(res,city):
    starbucks = []
    for r in res['results']:
        starbucks.append({
            'classification':'Starbucks',
            'address':r['formatted_address'],
            'city': city,
            'country':r['formatted_address'].split(', ')[-1],
            'coordinates':[*r['geometry']['location'].values()],
            'rating':r['rating']
        })
    return starbucks

for city in selected_cities:
    res = googleQuery(f'Starbucks in {city}')
    db['city_study'].insert_many(starbucksToMongo(res,city))
        print(starbucksToMongo(res.json(),city))
        res = googleQuery(f'Starbucks in {city}',next_page_token)
        next_page_token = res.json().get('next_page_token',None)
    print(starbucksToMongo(res.json(),city))
    #db['city_study'].insert_many(starbucksToMongo(res,city))    


Cambridge
Starbucks in Cambridge


NameError: name 'googleQuery' is not defined

In [205]:
next_page_token = res.json().get('next_page_token',None)
next_page_token

'CrQDqQEAAFPpCKXQa85ZsUXhAxu9Jxx-0P6wVb_bhWdw9L4LwmVzl2vCQGLjpUkPEfj0lTmTBBRdOZDkgBpalwzen5J-bJaTE3FJpz42XujoyH9kJkD-S_3qSPrQsHxL33O2gqhwVEX7y6zx0pmVvI3x3TRsWkk443hd6aas7TZgmkge1RHP4NpkufFqaLBKLyHrdgIRTJf449lDIg25-VauBS7J2diIP_NQVYeNKOWxR6ohAp4i7lJKC1qFfkwMBYwo5nXOpLGugvDBzbzV3pJNgNjrTlJhirm7NLxcCpA6s3BB0ygkBKjBJThVUX2Id6pFpr0K12LMFeJQhmPxlp5gzAMUvCVf9sy61ybASoRTJzxknsYfUo8QtAUrrdcvpWm7wPbL8Il3POGp3rh2aZQ8eTruO2aPYG13VMecBk3M-aRSj6GBYsId7ubByI2hXqr6yQAnhH04BSVH9jR_S61YSw_6dqBt5Pqark__H5LJ32j-aPHzRRiU_Y-NBmLja-0Jrl_4z1P6FNHNDT6r2PdWHV00BZ3yjuGTKQPXVCrBRiQ7iQ-aCfUnvu5_wi6wjasqbHeZARIQKTAKOyuEPxDOJPtJ2D5OahoUggGR5UiJNOmBabtLDTtUusMQbS8'

In [199]:
def googleQuery(query,pagetoken):
    authToken = f.authorizationKey('GOOGLE_TOKEN')
    url = f'https://maps.googleapis.com/maps/api/place/textsearch/json?query={query}&pagetoken={pagetoken}&key={authToken}'
    return requests.get(url)

res3 = googleQuery('Starbucks in San Francisco',next_page_token)

We have a github token:  AIza
